# **Convolutional Neural Network**
**Starting parameters**:

- Number of epochs: 100
- Learning rate: 0.0005
- Layers: [64, 32, 32]
- Dropout: 0.3


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For preprocessing
import tensorflow as tf

# For modeling
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random

In [60]:
pkl_path_2D = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated Test/test_features_3s_all_2D.pkl'

# Load the pickle file
with open(pkl_path_2D, 'rb') as file:
    test_2D = pickle.load(file)
del file

In [61]:
test_f_2D = test_2D['test'].copy()
test_labels = test_f_2D['label'].copy()

temp = test_f_2D.copy()
del temp['label']
test_features_2D = temp

## **Shuffling Data**

In [62]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [63]:
test_y_2D, test_X_2D = shuffle_data(test_labels, test_features_2D)

## **CNN Model**

In [64]:
def tile_and_crop(feature, target_size):
    tiled = np.tile(feature, (1, target_size // feature.shape[1] + 1, 1))
    return tiled[:, :target_size, :]

In [65]:
test_results = {}

# **Deep Learning Models**

In [66]:
# 2D

ffnn_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/FFNN/FFNN_melspectrogram_cqt_mfcc.keras')
cnn_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/CNN/CNN_melspectrogram_cqt_mfcc.keras')
EB0_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/EfficientB0_melspectrogram_cqt_mfcc.keras')
EB4_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/EfficientB4_melspectrogram_cqt_mfcc.keras')
Res_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/ResNet50_melspectrogram_cqt_mfcc.keras')
VGG_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/VGG16_melspectrogram_cqt_mfcc.keras')

ffnn_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/FFNN/FFNN_melspectrogram_chroma_mfcc.keras')
cnn_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/CNN/CNN_melspectrogram_chroma_mfcc.keras')
EB0_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/EfficientB0_melspectrogram_chroma_mfcc.keras')
EB4_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/EfficientB4_melspectrogram_chroma_mfcc.keras')
Res_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/ResNet50_melspectrogram_chroma_mfcc.keras')
VGG_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/VGG16_melspectrogram_chroma_mfcc.keras')

# **Preparing Test Data**

## 2D Testing Stuff

In [67]:
target_size = 128
mfcc_tiled = tile_and_crop(test_X_2D['mfcc'], target_size)
chroma_tiled = tile_and_crop(test_X_2D['chroma'], target_size)
cqt_tiled = tile_and_crop(test_X_2D['cqt'], target_size)

testing_2D_melmfccchroma = np.stack((test_X_2D['melspectrogram'], mfcc_tiled, chroma_tiled), axis=-1)
testing_2D_melmfcccqt = np.stack((test_X_2D['melspectrogram'], mfcc_tiled, cqt_tiled), axis=-1)

# **Evaluation**

Mel Spectrogram/MFCC/CQT Models

In [68]:
accuracy = cnn_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['CNN_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7357 - loss: 1.0607 - weighted_accuracy: 0.7357
Test Accuracy: 0.7254740595817566


In [69]:
accuracy = EB0_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['EB0_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.0214 - loss: 20587.1484
Test Accuracy: 0.024732070043683052


In [70]:
accuracy = EB4_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['EB4_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 32s 186ms/step - accuracy: 0.0973 - loss: 639.5522
Test Accuracy: 0.09645506739616394


In [71]:
accuracy = Res_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['Res_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.8224 - loss: 1.3818
Test Accuracy: 0.8124485015869141


In [72]:
accuracy = VGG_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['VGG_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 24s 190ms/step - accuracy: 0.8575 - loss: 0.7736
Test Accuracy: 0.8569661974906921


# **Evaluation**

Mel Spectrogram/MFCC/Chroma Models

In [73]:
accuracy = cnn_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['CNN_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8023 - loss: 0.8305 - weighted_accuracy: 0.8023
Test Accuracy: 0.7947238087654114


In [74]:
accuracy = EB0_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['EB0_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - accuracy: 0.0827 - loss: 486916.2812
Test Accuracy: 0.0915086567401886


In [75]:
accuracy = EB4_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['EB4_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 21s 124ms/step - accuracy: 0.0403 - loss: 84634.6562
Test Accuracy: 0.037098102271556854


In [76]:
accuracy = Res_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['Res_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 11s 75ms/step - accuracy: 0.8630 - loss: 0.8294
Test Accuracy: 0.8499587774276733


In [77]:
accuracy = VGG_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['VGG_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - accuracy: 0.8943 - loss: 0.5201
Test Accuracy: 0.8924155235290527


## Review all the results

In [80]:
test_results_df = pd.DataFrame(list(test_results.items()), columns=['Models', 'Testing_Accuracy']).round(3)

test_results_df = test_results_df.sort_values('Models')
test_results_df

,Models,Testing_Accuracy
0,CNN_CQT,0.725
5,CNN_Chroma,0.795
1,EB0_CQT,0.025
6,EB0_Chroma,0.092
2,EB4_CQT,0.096
7,EB4_Chroma,0.037
3,Res_CQT,0.812
8,Res_Chroma,0.850
4,VGG_CQT,0.857
9,VGG_Chroma,0.892
